<a href="https://colab.research.google.com/github/TejasJoshi2005/Modal-Analysis-of-Fluid-Flows/blob/main/Turbulent_flow_past_cylinders_for_verying_Re_number.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ==============================================================================
# Step 0: Import Necessary Libraries
# ==============================================================================
# We need these libraries to work with our data:
# - pandas: For loading and handling our data files.
# - numpy: For numerical operations, especially the math behind POD and DMD.
# - matplotlib.pyplot: For creating plots to visualize our results.
# - glob: To automatically find all of our data files in the directory.
# - os: To help with file path and name manipulation.

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import svd
import glob
import os

print("Libraries imported successfully!")

# ==============================================================================
# Step 1: Define the Main Analysis Function
# ==============================================================================
# We will create a primary function to handle the entire analysis for a single file.
# This makes our code clean and allows us to easily loop through all your data files later.

def process_and_analyze_flow_data(file_path, show_plots=True):
    """
    This function takes the path to a velocity data file and performs a full
    POD and DMD analysis, printing and plotting the results.
    """
    file_name = os.path.basename(file_path)
    print(f"\n{'='*60}")
    print(f"🚀 Starting Analysis for: {file_name}")
    print(f"{'='*60}")

    # --- Part A: Load and Prepare the Data ---
    print("\n[Part A] Loading and preparing the data...")
    try:
        # Load the text file into a pandas DataFrame.
        # The data is space-separated, has no header, and we'll name the columns 'time' and 'velocity'.
        data = pd.read_csv(file_path, sep='\s+', header=None, names=['time', 'velocity'])
        signal = data['velocity'].values  # This is our main 1D data signal (a numpy array)
        time = data['time'].values
        dt = time[1] - time[0]  # Calculate the time step between measurements
        print(f"Data loaded successfully. Found {len(signal)} data points.")
        print(f"Time step (dt) is: {dt:.4f} seconds (Sampling Frequency: {1/dt:.0f} Hz)")
    except Exception as e:
        print(f"Error loading file {file_name}: {e}")
        return None # Stop analysis for this file if it can't be loaded

    # --- Part B: Create the Snapshot Matrix ---
    # Both POD and DMD require the 1D time series data to be converted into a 2D matrix.
    # We do this by creating a "sliding window" over the signal.
    print("\n[Part B] Creating the snapshot matrix...")
    window_size = 1000  # How many data points in each snapshot. This is a key parameter.
    n_snapshots = len(signal) - window_size + 1

    # Create the matrix 'X' where each column is a snapshot of the signal.
    X = np.array([signal[i:i + window_size] for i in range(n_snapshots)]).T
    print(f"Snapshot matrix 'X' created with shape: {X.shape} (window_size x n_snapshots)")


    # --- Part C: Proper Orthogonal Decomposition (POD) ---
    # POD finds the dominant spatial patterns (modes) in the data.
    print("\n[Part C] Performing Proper Orthogonal Decomposition (POD)...")

    # The core of POD is the Singular Value Decomposition (SVD).
    # U: Contains the POD modes (the "spatial shapes").
    # S: Contains the singular values (the "energy" of each mode).
    # Vt: Contains the temporal evolution of each mode.
    U, S, Vt = svd(X, full_matrices=False)

    print("SVD computed for POD.")

    if show_plots:
        # Plot the energy of each mode. You'll typically see that the first few modes
        # contain almost all of the energy of the system.
        plt.figure(figsize=(12, 6))
        plt.semilogy(S / np.sum(S), 'o-', color='blue')
        plt.title(f'POD Energy Spectrum for {file_name}')
        plt.xlabel('Mode Number')
        plt.ylabel('Normalized Singular Value (Energy)')
        plt.grid(True)
        plt.show()

    # --- Part D: Dynamic Mode Decomposition (DMD) ---
    # DMD finds the dominant dynamic behaviors (modes that oscillate and grow/decay at a certain rate).
    print("\n[Part D] Performing Dynamic Mode Decomposition (DMD)...")

    # DMD works by finding a linear mapping from the state at one time to the next.
    # So, we create two matrices: X1 (current states) and X2 (next states).
    X1 = X[:, :-1]
    X2 = X[:, 1:]

    # Step 1 of DMD: Decompose the first matrix X1 using SVD.
    U_dmd, S_dmd, Vt_dmd = svd(X1, full_matrices=False)

    # Step 2: Calculate the DMD operator 'A' that approximates X2 ≈ A @ X1.
    # We compute a low-rank version called A_tilde.
    A_tilde = U_dmd.T @ X2 @ Vt_dmd.T @ np.diag(1./S_dmd)

    # Step 3: Find the eigenvalues and eigenvectors of this operator.
    # The eigenvalues tell us about the frequency and stability of the modes.
    eigenvalues, eigenvectors = np.linalg.eig(A_tilde)
    print("DMD operator computed and eigenvalues found.")

    # Step 4: Reconstruct the DMD modes.
    dmd_modes = X2 @ Vt_dmd.T @ np.diag(1./S_dmd) @ eigenvectors

    # Calculate the continuous-time eigenvalues (frequencies and growth rates).
    omegas = np.log(eigenvalues) / dt
    frequencies = np.imag(omegas) / (2 * np.pi)  # The oscillation frequencies in Hz.
    growth_rates = np.real(omegas)               # The growth/decay rates.

    # Find the dominant frequency (likely the vortex shedding frequency).
    # We ignore very low frequencies and look for the mode with the highest energy.
    valid_indices = np.where(np.abs(frequencies) > 1)[0]
    dominant_frequency = 0
    if len(valid_indices) > 0:
        mode_amplitudes = np.linalg.norm(dmd_modes, axis=0)
        dominant_mode_idx = valid_indices[np.argmax(mode_amplitudes[valid_indices])]
        dominant_frequency = frequencies[dominant_mode_idx]
        print(f"✅ Dominant Frequency Found: {dominant_frequency:.2f} Hz")
    else:
        print("⚠️ Could not identify a clear dominant frequency.")


    if show_plots:
        # Plot the DMD eigenvalues on the complex plane.
        # This is the classic DMD plot. Modes on the red circle are stable.
        plt.figure(figsize=(8, 8))
        plt.scatter(np.real(eigenvalues), np.imag(eigenvalues), c=np.abs(frequencies), alpha=0.7)
        unit_circle = plt.Circle((0,0), 1, color='r', fill=False, linestyle='--')
        plt.gca().add_artist(unit_circle)
        plt.title(f'DMD Eigenvalues for {file_name}')
        plt.xlabel('Real Part')
        plt.ylabel('Imaginary Part')
        plt.axis('equal')
        plt.grid(True)
        plt.show()

    return file_name, dominant_frequency

# ==============================================================================
# Step 2: Run the Analysis for All Your Files
# ==============================================================================

# Use glob to automatically find all files that match the pattern "Velocity_*.txt".
# This is much better than typing each file name manually.
all_files = glob.glob('Velocity_*.txt')
all_files.sort()

# A dictionary to store the final results for our summary table.
results_summary = {}

if not all_files:
    print("❌ No data files found! Make sure your 'Velocity_*.txt' files are in the correct directory.")
else:
    print(f"\nFound {len(all_files)} files. Starting batch analysis...")
    # Loop through each file path and run our analysis function.
    for file_path in all_files:
        # Set show_plots=False if you don't want to see the plots for every single file.
        file_name, freq = process_and_analyze_flow_data(file_path, show_plots=True)
        if freq is not None:
            results_summary[file_name] = freq

# ==============================================================================
# Step 3: Display the Final Summary
# ==============================================================================

print("\n\n--- 📊 FINAL SUMMARY ---")
if results_summary:
    print(f"{'File Name':<30} | {'Dominant Frequency (Hz)':<30}")
    print("-" * 62)
    for name, freq in results_summary.items():
        print(f"{name:<30} | {freq:.2f}")
    print("-" * 62)
else:
    print("No results to display. Please check for errors during analysis.")

In [ ]:
import numpy as np
import pandas as pd
from scipy.linalg import svd
import glob
import os

def analyze_dmd_results(file_path):
    """
    Analyzes a velocity data file using DMD and returns the dominant frequency
    and its corresponding modal amplitude.
    """
    try:
        # --- 1. Load Data ---
        data = pd.read_csv(file_path, sep='\s+', header=None, names=['time', 'velocity'])
        signal = data['velocity'].values
        dt = data['time'].iloc[1] - data['time'].iloc[0]

        # --- 2. Create Snapshot Matrix ---
        window_size = 1000
        X = np.array([signal[i:i + window_size] for i in range(len(signal) - window_size + 1)]).T

        # --- 3. Perform DMD ---
        X1, X2 = X[:, :-1], X[:, 1:]
        U, S, Vt = svd(X1, full_matrices=False)
        A_tilde = U.T @ X2 @ Vt.T @ np.diag(1./S)
        eigenvalues, eigenvectors = np.linalg.eig(A_tilde)

        # Reconstruct the DMD modes (Phi)
        dmd_modes = X2 @ Vt.T @ np.diag(1./S) @ eigenvectors

        # --- 4. Calculate Frequencies ---
        frequencies = np.imag(np.log(eigenvalues) / dt) / (2 * np.pi)

        # --- 5. Calculate Modal Amplitudes ---
        # The amplitudes (b) are calculated by projecting the initial state (x0) onto the DMD modes.
        x0 = X[:, 0]
        b = np.linalg.pinv(dmd_modes) @ x0

        # --- 6. Find the Dominant Mode and its Properties ---
        # We look for the mode with the highest amplitude, ignoring near-zero frequencies.
        valid_indices = np.where(np.abs(frequencies) > 1)[0]
        if len(valid_indices) == 0:
            return None, None

        # Find the index of the mode with the maximum amplitude 'b'
        dominant_mode_idx = valid_indices[np.argmax(np.abs(b[valid_indices]))]

        dominant_frequency = frequencies[dominant_mode_idx]
        modal_amplitude = np.abs(b[dominant_mode_idx])

        return dominant_frequency, modal_amplitude

    except Exception as e:
        print(f"Could not process {os.path.basename(file_path)}: {e}")
        return None, None

# ==============================================================================
# Main script to run the analysis on all files
# ==============================================================================

# Find all data files in the directory
all_files = sorted(glob.glob('Velocity_*.txt'))
results = []

if not all_files:
    print("❌ No data files found! Make sure your '.txt' files are present.")
else:
    print(f"Found {len(all_files)} files. Starting analysis...")
    # Loop through each file and store the results
    for path in all_files:
        freq, amp = analyze_dmd_results(path)
        if freq is not None and amp is not None:
            results.append({
                "File Name": os.path.basename(path),
                "Dominant Frequency (Hz)": freq,
                "Modal Amplitude": amp
            })

# --- Print Final Summary Table ---
print("\n\n--- 📊 FINAL RESULTS: Frequency and Amplitude ---")

if results:
    # Convert results to a pandas DataFrame for nice printing
    results_df = pd.DataFrame(results)
    print(results_df.to_string(index=False, float_format="%.2f"))
else:
    print("No results to display. Please check for errors during analysis.")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.linalg import svd
import os

# --- 1. Load the Data ---
file_path = 'Velocity_1C50_16ms.txt'
print(f"Analyzing file: {os.path.basename(file_path)}")

data = pd.read_csv(file_path, sep='\s+', header=None, names=['time', 'velocity'])
signal = data['velocity'].values
time = data['time'].values

# --- 2. Create the Snapshot Matrix ---
window_size = 1000
n_snapshots = len(signal) - window_size + 1
X = np.array([signal[i:i + window_size] for i in range(n_snapshots)]).T

# --- 3. Perform POD (SVD) ---
# U: POD modes (spatial shapes)
# S: Singular values (energy of each mode)
# Vt: Time evolution of each mode
U, S, Vt = svd(X, full_matrices=False)
print("POD calculation complete.")

# --- 4. Calculate the Time Coefficients ---
# The actual time coefficients are the rows of Vt scaled by their corresponding singular value.
# coeffs = np.diag(S) @ Vt
# However, for visualization, plotting the rows of Vt is sufficient and standard.
time_coeffs = Vt

# --- 5. Create the Corresponding Time Vector ---
# The time coefficients correspond to each snapshot, not each original data point.
time_vector = time[:n_snapshots]

# --- 6. Plot the Results ---
plt.figure(figsize=(15, 8))

# Plot the time coefficients for the first 4 modes
for i in range(4):
    plt.plot(time_vector, time_coeffs[i, :], label=f'Mode {i+1} Amplitude')

plt.title('Modal Amplitude (Time Coefficients) vs. Time', fontsize=16)
plt.xlabel('Time (s)', fontsize=12)
plt.ylabel('Modal Amplitude (Arbitrary Units)', fontsize=12)
plt.legend()
plt.grid(True)
plt.xlim(0, 15) # Zoom in on the first 15 seconds to see the oscillations clearly
plt.show()

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.linalg import svd
import glob
import os

# --- 1. Find all the velocity data files ---
all_files = sorted(glob.glob('Velocity_*.txt'))

if not all_files:
    print("❌ No data files found! Make sure your '.txt' files are in the correct directory.")
else:
    print(f"Found {len(all_files)} files. Generating plots for each...")

    # --- 2. Loop through each file and create a plot ---
    for file_path in all_files:
        print(f"\n--- Processing: {os.path.basename(file_path)} ---")

        # --- Load the data for the current file ---
        data = pd.read_csv(file_path, sep='\s+', header=None, names=['time', 'velocity'])
        signal = data['velocity'].values
        time = data['time'].values

        # --- Create the Snapshot Matrix ---
        window_size = 1000
        n_snapshots = len(signal) - window_size + 1
        X = np.array([signal[i:i + window_size] for i in range(n_snapshots)]).T

        # --- Perform POD (SVD) ---
        U, S, Vt = svd(X, full_matrices=False)

        # Vt contains the time evolution of each mode
        time_coeffs = Vt

        # --- Create the Corresponding Time Vector ---
        time_vector = time[:n_snapshots]

        # --- Plot the Results for the current file ---
        plt.figure(figsize=(15, 8))

        # Plot the time coefficients for the first 4 modes
        for i in range(4):
            plt.plot(time_vector, time_coeffs[i, :], label=f'Mode {i+1} Amplitude')

        # Use the file name in the title
        plt.title(f'Modal Amplitude vs. Time for {os.path.basename(file_path)}', fontsize=16)
        plt.xlabel('Time (s)', fontsize=12)
        plt.ylabel('Modal Amplitude (Arbitrary Units)', fontsize=12)
        plt.legend()
        plt.grid(True)
        plt.xlim(0, 15)  # Zoom in on the first 15 seconds to see details
        plt.show()

print("\n✅ All files have been processed and plotted.")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.linalg import svd
import glob
import os

# --- 1. Set the Plot Style to "Dark Mode" ---
plt.style.use('dark_background')

# --- 2. Define a function to create the "glow" effect ---
def plot_glow_effect(ax, x, y, color, label, n_glow_lines=10):
    """
    Plots a line with a soft glow effect on the given axes.
    """
    # Draw several faint, thick lines underneath the main line to create the glow
    for i in range(n_glow_lines):
        ax.plot(x, y, linewidth=2 + i * 1.5, color=color, alpha=0.05)
    # Draw the main, bright line on top
    ax.plot(x, y, linewidth=1.5, color=color, label=label)


# --- 3. Find all the velocity data files ---
all_files = sorted(glob.glob('Velocity_*.txt'))

if not all_files:
    print("❌ No data files found! Make sure your '.txt' files are in the correct directory.")
else:
    print(f"Found {len(all_files)} files. Generating stylized plots for each...")

    # Define a list of neon colors for the different modes
    neon_colors = ['#08F7FE', '#FE53BB', '#F5D300', '#00ff41']

    # --- 4. Loop through each file and create a plot ---
    for file_path in all_files:
        print(f"\n--- Processing: {os.path.basename(file_path)} ---")

        # Load the data for the current file
        data = pd.read_csv(file_path, sep='\s+', header=None, names=['time', 'velocity'])
        signal = data['velocity'].values
        time = data['time'].values

        # Perform POD (SVD)
        window_size = 1000
        n_snapshots = len(signal) - window_size + 1
        X = np.array([signal[i:i + window_size] for i in range(n_snapshots)]).T
        U, S, Vt = svd(X, full_matrices=False)
        time_coeffs = Vt
        time_vector = time[:n_snapshots]

        # --- 5. Create the Stylized Plot ---
        fig, ax = plt.subplots(figsize=(15, 8))

        # Plot the time coefficients for the first 4 modes with the glow effect
        for i in range(4):
            plot_glow_effect(ax, time_vector, time_coeffs[i, :], color=neon_colors[i], label=f'Mode {i+1}')

        # --- 6. Customize the plot appearance ---
        ax.set_title(f'Modal Amplitude vs. Time for {os.path.basename(file_path)}', fontsize=18, color='white')
        ax.set_xlabel('Time (s)', fontsize=14, color='white')
        ax.set_ylabel('Modal Amplitude', fontsize=14, color='white')

        # Customize ticks and legend
        ax.tick_params(axis='x', colors='white')
        ax.tick_params(axis='y', colors='white')
        legend = ax.legend()
        for text in legend.get_texts():
            text.set_color('white')

        # Remove the grid and make axis lines white
        ax.grid(False)
        for spine in ax.spines.values():
            spine.set_edgecolor('white')

        ax.set_xlim(0, 15) # Zoom in on the first 15 seconds
        plt.show()

print("\n✅ All files have been processed and plotted.")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.linalg import svd
import glob
import os

# --- 1. Find all the velocity data files ---
all_files = sorted(glob.glob('Velocity_*.txt'))

if not all_files:
    print("❌ No data files found! Make sure your '.txt' files are in the correct directory.")
else:
    print(f"Found {len(all_files)} files. Generating pcolormesh plots for each...")

    # --- 2. Loop through each file and create a plot ---
    for file_path in all_files:
        print(f"\n--- Processing: {os.path.basename(file_path)} ---")

        # --- Load the data ---
        data = pd.read_csv(file_path, sep='\s+', header=None, names=['time', 'velocity'])
        signal = data['velocity'].values
        time = data['time'].values

        # --- Create the Snapshot Matrix ---
        window_size = 500 # Using a slightly smaller window for better visualization
        n_snapshots = len(signal) - window_size + 1
        X = np.array([signal[i:i + window_size] for i in range(n_snapshots)]).T

        # --- Perform POD (SVD) ---
        U, S, Vt = svd(X, full_matrices=False)

        # --- Reconstruct the data using the first 6 dominant modes ---
        # This highlights the main coherent structures and filters noise.
        n_modes_to_use = 6
        X_reconstructed = U[:, :n_modes_to_use] @ np.diag(S[:n_modes_to_use]) @ Vt[:n_modes_to_use, :]

        # Subtract the mean to focus on fluctuations (vortices)
        X_reconstructed = X_reconstructed - np.mean(X_reconstructed)

        # --- Create Coordinate Vectors for the Plot ---
        time_vector = time[:n_snapshots]
        space_vector = np.arange(window_size)

        # --- Create the pcolormesh Plot ---
        plt.figure(figsize=(16, 8))

        # Use a diverging colormap like 'seismic' or 'RdBu' which is great for flow fields
        # It shows positive fluctuations in one color and negative in another.
        plt.pcolormesh(time_vector, space_vector, X_reconstructed, cmap='seismic', shading='gouraud')

        # --- Customize the Plot ---
        plt.colorbar(label='Velocity Fluctuation (m/s)')
        plt.title(f'Reconstructed Flow Dynamics for {os.path.basename(file_path)}', fontsize=16)
        plt.xlabel('Time (s)', fontsize=12)
        plt.ylabel('Position in Snapshot Window', fontsize=12)
        plt.xlim(0, 20) # Zoom in on the first 20 seconds to see the structure

        plt.show()

print("\n✅ All files have been processed and plotted.")